# Building a Model for flower detection app

In [ ]:
import tensorflow as tf
tf.__version__

'2.4.1'

## Download the dataset

In [ ]:
dataset_url = 'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz'
zip_file = tf.keras.utils.get_file(origin=dataset_url, fname='flower_dataset.tgz', extract=True, cache_subdir='/content')

228818944/228813984 [==============================] - 3s 0us/step


## Creating Data Generators for Training

In [ ]:
base_dir = '/content/flower_photos' # Base directory

In [ ]:
IMAGE_SIZE = 224 
BATCH_SIZE = 64 

In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1/255.,
    validation_split=0.2
) 

# Train 
train_generator = datagen.flow_from_directory(
    base_dir,
    target_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    subset = 'training'
)

# Validation
val_generator = datagen.flow_from_directory(
    base_dir,
    target_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    subset = 'validation'
)

Found 2939 images belonging to 5 classes.
Found 731 images belonging to 5 classes.


## Creating our labels file for classes

In [ ]:
labels = '\n'.join(sorted(train_generator.class_indices.keys()))
print(labels)

daisy
dandelion
roses
sunflowers
tulips


In [ ]:
with open('labels.txt', 'w') as f:
    f.write(labels)

## Building a Model

Here, we will use transfer learning to make our predictions accurate. We are gonna use MobileNetV2 model as our base model and build our own model on top of it.

In [ ]:
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
base_model.trainable = False # We dont want to train mobilenet layers

In [ ]:
model = tf.keras.Sequential([
                             base_model,
                             tf.keras.layers.Conv2D(64, 3, activation='relu'),
                             tf.keras.layers.MaxPool2D((2,2)),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(64, activation='relu'),
                             tf.keras.layers.Dropout(0.2),
                             tf.keras.layers.Dense(5, activation='softmax')
])

In [ ]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 5, 5, 64)          737344    
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 2, 2, 64)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 2, 2, 64)          256       
_________________________________________________________________
flatten_8 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_28 (Dense)             (None, 64)                16448     
_________________________________________________________________
dropout_18 (Dropout)         (None, 64)              

## Compile and Fit the model
We will use Adam as our optimizer, also we have 5 different classes so we are gonna use categorical cross entropy.

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Fitting the model
EPOCHS = 5 

history = model.fit(train_generator,
                    epochs=EPOCHS,
                    validation_data=val_generator)

Epoch 1/5
46/46 [==============================] - 20s 361ms/step - loss: 0.7804 - accuracy: 0.7073 - val_loss: 0.4775 - val_accuracy: 0.8523
Epoch 2/5
46/46 [==============================] - 15s 334ms/step - loss: 0.2121 - accuracy: 0.9289 - val_loss: 0.4167 - val_accuracy: 0.8700
Epoch 3/5
46/46 [==============================] - 15s 332ms/step - loss: 0.0872 - accuracy: 0.9773 - val_loss: 0.4617 - val_accuracy: 0.8440
Epoch 4/5
46/46 [==============================] - 15s 328ms/step - loss: 0.0404 - accuracy: 0.9912 - val_loss: 0.4840 - val_accuracy: 0.8427
Epoch 5/5
46/46 [==============================] - 15s 328ms/step - loss: 0.0263 - accuracy: 0.9954 - val_loss: 0.4941 - val_accuracy: 0.8659


## Saving the model

In [ ]:
# Saving the model
tf.saved_model.save(model, '/content/models')

INFO:tensorflow:Assets written to: /content/models/assets


## Convert the saved model to tflite format so that we can use it in our app

In [ ]:
# Convert the model to tflite
converter = tf.lite.TFLiteConverter.from_saved_model('/content/models')

tflite_model = converter.convert()

In [ ]:
with open('flower_model.tflite', 'wb') as f:
    f.write(tflite_model)

## Finally Download the model

In [ ]:
from google.colab import files
files.download('flower_model.tflite')
files.download('labels.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>